In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
from sklearn.model_selection import ParameterGrid, train_test_split
import numpy as np

In [16]:
# 1. Параметры (теперь некоторые из них будем перебирать Grid Search)
sequence_length = 10  # Длина последовательности
# num_sequences = 1000 # Количество последовательностей для обучения  (зададим в Grid Search)
# hidden_size = 32  # Размерность скрытого слоя (зададим в Grid Search)
# learning_rate = 0.1  # Скорость обучения для оптимизаторa (зададим в Grid Search)
num_epochs = 100 # Уменьшим количество эпох, потому что будем много обучать
num_digits = 10 # от 0 до 9

num_layers = 1

In [17]:
def generate_data(num_sequences, sequence_length):
    X = torch.randint(0, num_digits, (num_sequences, sequence_length))  # Случайные последовательности цифр
    Y = torch.zeros_like(X) # Создаем тензор той же формы, что и X
    for i in range(num_sequences):
        x1 = X[i, 0].item() # Первый элемент последовательности x
        Y[i, 0] = x1  # y1 = x1
        for j in range(1, sequence_length):
            yi = X[i, j].item() + x1 #  yi = xi + x1
            if yi >= num_digits:  # Если yi >= 10
                yi -= num_digits  # yi = yi - 10
            Y[i, j] = yi
    return X.long(), Y.long() # Возвращаем long тензоры

In [18]:
class DigitRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, rnn_type='RNN', num_layers=1): # Добавили num_layers
        super(DigitRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn_type = rnn_type
        self.num_layers = num_layers # Сохраняем num_layers
        if rnn_type == 'RNN':
            self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        elif rnn_type == 'LSTM':
            self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        else:
            raise ValueError("Неподдерживаемый тип RNN")
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Инициализация скрытого состояния
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)  # (num_layers, batch_size, hidden_size)

        if self.rnn_type == 'LSTM':
            c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device) # Добавили num_layers
            out, _ = self.rnn(x, (h0, c0))
        elif self.rnn_type == 'GRU':
            out, _ = self.rnn(x, h0) # GRU не принимает c0
        else:  # RNN
            out, _ = self.rnn(x, h0)

        out = self.linear(out)  # Пропускаем через полносвязный слой
        return out
    

In [19]:
# 4. Функция обучения модели (слегка модифицирована)
def train_model(model, X, Y, learning_rate, num_epochs, batch_size):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Создаем DataLoader для пакетной обработки (batching)
    data = torch.utils.data.TensorDataset(X, Y)
    data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)

    for epoch in range(num_epochs):
        model.train()
        for X_batch, Y_batch in data_loader: # Итерируемся по батчам
            optimizer.zero_grad() # Обнуляем градиенты

            # One-hot encode входные данные
            X_onehot = torch.nn.functional.one_hot(X_batch, num_classes=num_digits).float()

            # Forward pass
            outputs = model(X_onehot)  # (batch_size, sequence_length, num_digits)

            # Вычисляем loss
            loss = criterion(outputs.view(-1, num_digits), Y_batch.view(-1))  # Изменяем форму

            # Backpropagation и оптимизация
            loss.backward() # oбратное распространение ошибки
            optimizer.step() # обновляем параметры модели

        # Выводим loss только в конце каждой эпохи
        print(f'Эпоха [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    return model  # Возвращаем обученную модель


In [20]:
# Grid Search
# Определяем сетку параметров
param_grid = {
    'num_sequences': [500, 1000], # Количество обучающих последовательностей
    'hidden_size': [32, 64], # Размер скрытого слоя
    'learning_rate': [0.01, 0.001], # Скорость обучения
    'rnn_type': ['RNN', 'LSTM', 'GRU'], # Тип рекуррентной сети
    'batch_size': [32, 64] # Размер батча
}

In [21]:
# Создаем все комбинации параметров
grid = ParameterGrid(param_grid)

best_model = None
best_loss = float('inf')
best_params = None


# Цикл по всем комбинациям параметров
for params in grid:
    print(f"Текущие параметры: {params}")

    # Генерируем данные с текущим num_sequences
    X, Y = generate_data(params['num_sequences'], sequence_length)

    # Определяем batch_size
    batch_size = params['batch_size']

    # Разбиваем данные на train и validation
    X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)  # Валидация

    # Создаем модель с текущими гиперпараметрами
    model = DigitRNN(input_size=num_digits,
                       hidden_size=params['hidden_size'],
                       output_size=num_digits,
                       rnn_type=params['rnn_type'],
                       num_layers=num_layers) # ВАЖНО: Передаем num_layers

    # Обучаем модель
    model = train_model(model, X_train, Y_train, params['learning_rate'], num_epochs, batch_size)

    # Оцениваем модель на валидационной выборке
    model.eval()  # Переводим модель в режим оценки

    # One-hot encode входные данные
    X_val_onehot = torch.nn.functional.one_hot(X_val, num_classes=num_digits).float()

    # Forward pass на валидационной выборке
    with torch.no_grad():  # Отключаем вычисление градиентов
        outputs = model(X_val_onehot)

    # Вычисляем loss на валидационной выборке
    criterion = nn.CrossEntropyLoss()
    loss = criterion(outputs.view(-1, num_digits), Y_val.view(-1))

    print(f"Validation Loss: {loss.item():.4f}")

    # Если текущая модель лучше, чем лучшая предыдущая, сохраняем ее
    if loss.item() < best_loss:
        best_loss = loss.item()
        best_model = model
        best_params = params
        print("Обновлена лучшая модель!")

   
# 6. Вывод результатов Grid Search
print("-----------------------------------")
print("Grid Search завершен!")
print(f"Лучшие параметры: {best_params}")
print(f"Лучший Validation Loss: {best_loss:.4f}")

# 7. Оценка лучшей модели на тестовой выборке 
# Можно догенерировать новые данные для теста, если это необходимо

# 8. Использование лучшей модели (пример)
#  предсказать выход для новой последовательности X_new:
# X_new = torch.randint(0, num_digits, (1, sequence_length))
# best_model.eval() # Переводим в режим оценки
# X_new_onehot = torch.nn.functional.one_hot(X_new, num_classes=num_digits).float()
# with torch.no_grad():
#   prediction = best_model(X_new_onehot)
#   predicted_digits = torch.argmax(prediction, dim=2) #  Выбираем индекс с максимальной вероятностью
# print("Предсказанные цифры:", predicted_digits)


Текущие параметры: {'batch_size': 32, 'hidden_size': 32, 'learning_rate': 0.01, 'num_sequences': 500, 'rnn_type': 'RNN'}
Эпоха [1/100], Loss: 2.2897
Эпоха [2/100], Loss: 2.2819
Эпоха [3/100], Loss: 2.2826
Эпоха [4/100], Loss: 2.2646
Эпоха [5/100], Loss: 2.1856
Эпоха [6/100], Loss: 2.2598
Эпоха [7/100], Loss: 2.3182
Эпоха [8/100], Loss: 2.2718
Эпоха [9/100], Loss: 2.2852
Эпоха [10/100], Loss: 2.3128
Эпоха [11/100], Loss: 2.2050
Эпоха [12/100], Loss: 2.2184
Эпоха [13/100], Loss: 2.2134
Эпоха [14/100], Loss: 2.2073
Эпоха [15/100], Loss: 2.1634
Эпоха [16/100], Loss: 2.2184
Эпоха [17/100], Loss: 2.1543
Эпоха [18/100], Loss: 1.9666
Эпоха [19/100], Loss: 2.1811
Эпоха [20/100], Loss: 2.1449
Эпоха [21/100], Loss: 2.1075
Эпоха [22/100], Loss: 2.1488
Эпоха [23/100], Loss: 2.0989
Эпоха [24/100], Loss: 2.0895
Эпоха [25/100], Loss: 2.0425
Эпоха [26/100], Loss: 1.9115
Эпоха [27/100], Loss: 1.9101
Эпоха [28/100], Loss: 2.0903
Эпоха [29/100], Loss: 2.1291
Эпоха [30/100], Loss: 1.9265
Эпоха [31/100], Lo

In [23]:
# 6. Вывод результатов Grid Search
print("-----------------------------------")
print("Grid Search завершен!")
print(f"Лучшие параметры: {best_params}")
print(f"Лучший Validation Loss: {best_loss:.4f}")



-----------------------------------
Grid Search завершен!
Лучшие параметры: {'batch_size': 32, 'hidden_size': 64, 'learning_rate': 0.01, 'num_sequences': 1000, 'rnn_type': 'LSTM'}
Лучший Validation Loss: 0.0002


In [24]:
# Сгенерируем *новую* входную последовательность для предсказания (как в вашем коде)
X_new, Y_new = generate_data(1, sequence_length)  # 1 последовательность длиной sequence_length

# Переведем модель в режим оценки (это важно для корректной работы некоторых слоев, например, BatchNorm и Dropout)
best_model.eval()

#  Преобразуем входную последовательность в one-hot encoding
X_new_onehot = torch.nn.functional.one_hot(X_new, num_classes=num_digits).float()

# 4. Отключим вычисление градиентов (это уменьшает потребление памяти и ускоряет вычисления во время предсказания)
with torch.no_grad():
    # 5. Получим выходные данные модели
    outputs = best_model(X_new_onehot)

    # 6. Преобразуем выходные данные в предсказанные цифры
    predicted_digits = torch.argmax(outputs, dim=2)  # Выбираем индекс с максимальной вероятностью

# 7. Выведем результаты
print("Входная последовательность (X_new):", X_new)
print("Ожидаемая последовательность (Y_new):", Y_new)  # Сгенерированная целевая переменная
print("Предсказанная последовательность:", predicted_digits)

Входная последовательность (X_new): tensor([[8, 8, 9, 7, 1, 4, 4, 0, 5, 8]])
Ожидаемая последовательность (Y_new): tensor([[8, 6, 7, 5, 9, 2, 2, 8, 3, 6]])
Предсказанная последовательность: tensor([[8, 6, 7, 5, 9, 2, 2, 8, 3, 6]])
